Demonstrate we can specify a simple interface to load data sources.

In [1]:
import numpy as np
import pandas as pd

%load_ext autoreload

In [2]:
%autoreload 2

from ldr import Schema, Filter, Selector
from ingresso import Sales0

In [3]:
sources = { "fx": [ "tests/media/gbp-usd.csv", Schema(desc = "fx") ], 
           "fx2": [ "tests/media/gbp-usd2.csv", Schema(desc = "fx-fxcm") ], 
           "sales": [ "tests/media/sales.csv", Schema(desc = "sales") ],  
           "weather": [ "tests/media/london.csv", Schema(desc = "weather") ],
           "weather2": [ "tests/media/metoffice.csv", Schema(desc = "weather-metoffice") ] }

In [4]:
filters = list(map( lambda x: Filter(x[0], x[1]), sources.values()))

In [5]:
def postprocess0(f0):
    f0.series("datetime", index=True)
    return f0.series("value")

In [6]:
series = list(map(lambda x: postprocess0(x), filters))

Finally, merge the series together, we pass the monthly Met-office series separately.

In [25]:
s0s = list(filter(lambda x: x.name != "weather-metoffice", series))
s1 = list(filter(lambda x: x.name == "weather-metoffice", series))
df = pd.DataFrame(s0s).transpose()
s0 = Sales0(df, metoffice=s1)
print(s0)
s0.constrain()
print(s0)

'Sales0: ()'
'Sales0: (fx, fx-fxcm, sales, weather)'


In [26]:
print(s0._cdf.describe())
print("series-length: {0:d}".format(len(s0._cdf)))
print(s0._cdf.index[[0, -1]])
Selector.nulls(s0._cdf)

               fx      fx-fxcm        sales      weather
count  788.000000  1578.000000  1864.000000  1773.000000
mean     1.366371     1.460032  1115.015558    12.029724
std      0.098996     0.143808   927.505853     5.406125
min      1.204800     1.199990     0.000000    -4.100000
25%      1.292975     1.322625   310.000000     7.900000
50%      1.338350     1.472945   852.500000    11.900000
75%      1.437975     1.570043  1805.250000    16.300000
max      1.577600     1.716680  4864.000000    27.900000
series-length: 1864
DatetimeIndex(['2013-06-11', '2018-07-18'], dtype='datetime64[ns]', name='dt0', freq=None)


,N,R
fx,1076,0.6
fx-fxcm,286,0.2
weather,91,0.0
sales,0,0.0


Resample to monthly, check all not null and write to CSV for R.

In [50]:
s0.fx()
s0s = list(map(lambda x: s0.resample(x), ['fx0', 'weather', 'sales']))
ts0s = list(map(lambda x: all(x.notnull()), s0s))
          
assert all(ts0s)

In [54]:
df0 = pd.DataFrame(s0s).transpose()
df0.to_csv("sales0-M.csv")